In [1]:
!nvidia-smi

Sun May 26 07:46:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA TITAN Xp                Off | 00000000:3B:00.0 Off |                  N/A |
| 23%   30C    P8               9W / 250W |      2MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/home/initial/CSP


In [4]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.22 🚀 Python-3.10.13 torch-2.3.0 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)
Setup complete ✅ (24 CPUs, 93.1 GB RAM, 2159.1/3724.2 GB disk)


In [6]:
import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.20.0


In [7]:
MODEL = "yolov8x.pt"
from ultralytics import YOLO
model = YOLO(MODEL)
model.fuse()

100%|██████████| 131M/131M [01:14<00:00, 1.84MB/s] 


YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients


In [15]:
print(list(model.model.names.keys())[list(model.model.names.values()).index("person")])
print(list(model.model.names.keys())[list(model.model.names.values()).index("bicycle")])
print(list(model.model.names.keys())[list(model.model.names.values()).index("motorcycle")])
print(list(model.model.names.keys())[list(model.model.names.values()).index("car")])
print(list(model.model.names.keys())[list(model.model.names.values()).index("bus")])
print(list(model.model.names.keys())[list(model.model.names.values()).index("truck")])



0
1
3
2
5
7


In [17]:
!gdown '1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-'

Downloading...
From: https://drive.google.com/uc?id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-
To: /home/initial/CSP/vehicle-counting.mp4
100%|██████████████████████████████████████| 35.3M/35.3M [00:04<00:00, 8.01MB/s]


In [18]:
SOURCE_VIDEO_PATH = f"{HOME}/vehicle-counting.mp4"

In [19]:
# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [2, 3, 5, 7]

In [20]:
import supervision as sv
import numpy as np

In [21]:
# settings
LINE_START = sv.Point(50, 1500)
LINE_END = sv.Point(3840-50, 1500)

TARGET_VIDEO_PATH = f"{HOME}/vehicle-counting-result-with-counter.mp4"

In [32]:
# create BYTETracker instance
byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30)

# create VideoInfo instance
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

# create LineZone instance, it is previously called LineCounter class
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)

# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)

# create instance of TraceAnnotator
trace_annotator = sv.TraceAnnotator(thickness=4, trace_length=50)

# create LineZoneAnnotator instance, it is previously called LineCounterAnnotator class
line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)

# define call back function to be used in video processing
def callback(frame: np.ndarray, index:int) -> np.ndarray:
    # model prediction on single frame and conversion to supervision Detections
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    # only consider class id from selected_classes define above
    detections = detections[np.isin(detections.class_id, selected_classes)]
    # tracking detections
    detections = byte_tracker.update_with_detections(detections)
    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]
    annotated_frame = trace_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )
    annotated_frame=box_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels)

    # update line counter
    line_zone.trigger(detections)
    # return frame with box and line annotated result
    return  line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

# process the whole video
sv.process_video(
    source_path = SOURCE_VIDEO_PATH,
    target_path = TARGET_VIDEO_PATH,
    callback=callback
)

SupervisionWarnings: `track_buffer` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'lost_track_buffer' instead.
SupervisionWarnings: `track_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'track_activation_threshold' instead.
SupervisionWarnings: `match_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'minimum_matching_threshold' instead.
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
Supe

In [33]:
class Counter:
    def __init__(self, start:tuple, end:tuple, video_file:str):
        self.start = start
        self.end = end
        self.video_file = video_file
        self.selected_classes = {'person':0, 'bicycle':1, 'motorcycle':3, 'car':2, 'bus':5, 'truck':7}
        self.byte_tracker = {'person': sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30),
                             'bicycle': sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30),
                             'motorcycle': sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30),
                             'car': sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30),
                             'bus': sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30),
                             'truck': sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30)}
        self.line_zones = {'person': sv.LineZone(start=sv.Point(*self.start), end=sv.Point(*self.end)),
                           'bicycle': sv.LineZone(start=sv.Point(*self.start), end=sv.Point(*self.end)),
                           'motorcycle': sv.LineZone(start=sv.Point(*self.start), end=sv.Point(*self.end)),
                           'car': sv.LineZone(start=sv.Point(*self.start), end=sv.Point(*self.end)),
                           'bus': sv.LineZone(start=sv.Point(*self.start), end=sv.Point(*self.end)),
                           'truck': sv.LineZone(start=sv.Point(*self.start), end=sv.Point(*self.end))}
        
    def process(self):
        def callback(frame: np.ndarray, index:int) -> np.ndarray:
            results = model(frame, verbose=False)[0]
            for key in self.selected_classes:
                detections = sv.Detections.from_ultralytics(results)
                detections = detections[detections.class_id == self.selected_classes[key]]
                detections = self.byte_tracker[key].update_with_detections(detections)
                self.line_zones[key].trigger(detections)
            return None
        
        sv.process_video(
            source_path = self.video_file,
            target_path = self.video_file.replace(".mp4", "-result.mp4"),
            callback=callback
        )
        return {key: self.line_zones[key].in_count + self.line_zones[key].out_count for key in self.line_zones}

In [34]:
Counter((50, 1500), (3840-50, 1500), SOURCE_VIDEO_PATH).process()

SupervisionWarnings: `track_buffer` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'lost_track_buffer' instead.
SupervisionWarnings: `track_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'track_activation_threshold' instead.
SupervisionWarnings: `match_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'minimum_matching_threshold' instead.
SupervisionWarnings: `track_buffer` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'lost_track_buffer' instead.
SupervisionWarnings: `track_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'track_activation_threshold' instead.
SupervisionWarnings: `match_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'minimum_matching_threshold' instead.
SupervisionWarnings: `track_buffer` in `ByteTrack.__init__

{'person': 0, 'bicycle': 0, 'motorcycle': 0, 'car': 6, 'bus': 0, 'truck': 0}